In [82]:
import pandas as pd
import numpy as np

import os
import glob

import matplotlib.pyplot as plt

import shutil
import ntpath

from sklearn.metrics import confusion_matrix
from scipy.stats import pearsonr

In [21]:
network_names = [
    'vgg16', 'vgg19',
    'densenet121', 'densenet169', 'densenet201',
    'mobilenet', 'mobilenet_v2',
    'nasnetmobile', 'nasnetlarge',
    'resnet50', 'inception_v3', 'inception_resnet_v2', 'xception',
]

experiments = ['KerasChromaticity', 'KerasRedGreen', 'KerasYellowBlue' , 'KerasLightness']

num_samples = 50
num_classes = 1000
num_images = num_samples * num_classes
gts = np.zeros((num_images, 1))
for i in range(num_classes):
    si = (i * num_samples)
    ei = si + num_samples
    gts[si:ei, 0] = i

In [74]:
def generate_confusions(input_folder, output_folder, gts):
    for experiment_input in glob.glob(input_folder + '/*/'):
        experiment_name = experiment_input.split('/')[-2]
        experiment_output = output_folder + '/' + experiment_name + '/'
        if not os.path.exists(experiment_output):
            os.mkdir(experiment_output)
        # going through networks
        for network_input in glob.glob(experiment_input + '/*/'):
            network_name = network_input.split('/')[-2]
            network_output = experiment_output + '/' + network_name + '/'
            if not os.path.exists(network_output):
                os.mkdir(network_output)
            for file in glob.glob(network_input + '*.csv'):
                experiment_predictions = np.loadtxt(file, delimiter=',')
                current_matrix = confusion_matrix(gts, experiment_predictions[:, 2])
                parameter_name = file.split('/')[-1]
                out_file = network_output + '/' + parameter_name
                print(out_file)
                np.savetxt(out_file, current_matrix, delimiter=',')

In [78]:
def do_one_network(input_folder, output_folder, network_name, experiments):
    file = input_folder + '/KerasOriginal/' + network_name + '/' + network_name + '_original.csv'
    original_predictions = np.loadtxt(file, delimiter=',')

    confusion_matrices = []
    for experiment in experiments:
        files = glob.glob('%s/%s/%s/*.csv' % (input_folder, experiment, network_name))
        output_folder = '%s/%s/' % (output_folder, experiment)
        if not os.path.exists(output_folder):
            os.mkdir(output_folder)
        output_folder = '%s/%s/' % (output_folder, network_name)
        if not os.path.exists(output_folder):
            os.mkdir(output_folder)
        for file in files:
            if '_0.0.csv' in file:
                experiment_predictions = np.loadtxt(file, delimiter=',')
                accuracy_drop = np.zeros((1000, 1))
                for i in range(1000):
                    accuracy_drop[i, 0] = experiment_predictions[i, i] / original_predictions[i, i]
                confusion_matrices.append(accuracy_drop)
                out_file = output_folder + 'accuracy_drop.csv'
                np.savetxt(out_file, accuracy_drop, delimiter=',')
    return confusion_matrices

In [172]:
input_folder = '/home/arash/Desktop/ICCV_Results/KerasConfusions/'
output_folder = '/home/arash/Desktop/ICCV_Results/KerasResults/'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

# generate_confusions(input_folder, output_folder, gts)

network_results = np.zeros((1000, 13))
for i, network_name in enumerate(network_names):
    experiments_results = do_one_network(input_folder, output_folder, network_name, experiments[2:3])
    network_results[:, i:i+1] = experiments_results[0]

In [103]:
net_corss = np.ones((13, 13))
suffix = '_red_green_0.0.csv'
folder = 'KerasRedGreen/'
for i in range(net_corss.shape[0] - 1):
    file_i = input_folder + folder + network_names[i] + '/' + network_names[i] + suffix
    org_result_i = np.loadtxt(file_i, delimiter=',')
    for j in range(i + 1, net_corss.shape[0]):
        file_j = input_folder + folder + network_names[j] + '/' + network_names[j] + suffix
        org_result_j = np.loadtxt(file_j, delimiter=',')
#         corr, p_value = pearsonr(network_results[:, i], network_results[:, j])
        corr, p_value = pearsonr(org_result_i.flatten(), org_result_j.flatten())
        net_corss[i, j] = corr
        net_corss[j, i] = corr
report_file = '/home/arash/Desktop/ICCV_Results/KerasReports/'
np.savetxt(report_file + 'red_green_confusion_corr.csv', net_corss,delimiter=',')

In [179]:
indices = np.where((network_results > 1).sum(axis=1) > 12)
print(len(imagenet_labels[indices]))
print(imagenet_labels[indices])

1
['barn spider']


In [156]:
imagenet_labels[10:14]

array(['brambling', 'goldfinch', 'house finch', 'junco'], dtype='<U32')

In [112]:
import json
label_file = '/home/arash/Desktop/ILSVRC2012_validation_labels.json'
with open(label_file) as f:
    imagenet_labels = json.load(f)
imagenet_labels = np.array(imagenet_labels)